# Set up a simple training loop

In [ ]:
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt; plt.style.use('dark_background')
π = np.pi
from numpy.random import uniform as uni
from tqdm import tqdm

In [ ]:
N_SAMPLES = 100

In [ ]:
# architecture
LATENT_DIM = 8
class Net(nn.Module):
    def __init__(self, in_dim, out_dim, hidden_dim=64):
        super(Net, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            # nn.ReLU(),
            # nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, out_dim)
        )
    def forward(self, x): return self.fc(x)

In [ ]:
# create a dataset
def create_random_signal(n_samples=N_SAMPLES, n_frqs=2):
    # generate random frequencies
    fs = uni(0, 5, n_frqs)
    As = uni(0.8, 1, n_frqs)
    φs = uni(0, 2*π, n_frqs)
    t = np.linspace(0, 1, n_samples)
    # generate the signal
    x = np.sum([As[i]*np.sin(2*π*fs[i]*t+φs[i]) for i in range(n_frqs)], axis=0).astype(np.float32)
    return x

class SigDS(Dataset):
    def __init__(self, n_ds):
        self.n_ds = n_ds
        self.data = torch.stack([torch.tensor(create_random_signal()) for _ in range(n_ds)])
    def __len__(self): return self.n_ds
    def __getitem__(self, idx): return self.data[idx]

ds = SigDS(20000)
dl = DataLoader(ds, batch_size=16, shuffle=True)

In [ ]:
# plot 10 random signals
plt.figure(figsize=(10, 10))
for i in range(10):
    plt.subplot(5, 2, i+1)
    plt.plot(ds[i])
plt.show()

In [ ]:
# define custom loss with custom gradient
class HLoss1(nn.Module):
    def __init__(self):
        super(HLoss1, self).__init__()
    def forward(self, x1, x2):
        x = x1 - x2
        b = F.softmax(x, dim=-1) * F.log_softmax(x, dim=-1)
        # b = -1.0 * b.sum() / x.size(0)
        b = b.sum() / x.size(0)
        return b

In [ ]:
# training loop
enc = Net(N_SAMPLES, LATENT_DIM)
dec = Net(LATENT_DIM, N_SAMPLES)
opt = optim.Adam(list(enc.parameters()) + list(dec.parameters()), lr=3e-3)

mse_loss = nn.MSELoss()
l1_loss = nn.L1Loss()
H_loss = HLoss1()

n_epochs = 20
lmses, lL1s, lHs = [], [], []
for epoch in (range(n_epochs)):
    elmse, elL1, elH = 0, 0, 0
    for x in dl:
        opt.zero_grad()

        z = enc(x)
        x̂ = dec(z)

        lmse = mse_loss(x̂, x)
        lL1 = l1_loss(x̂, x)
        lH = H_loss(x̂, x)

        elmse += lmse.item()
        elL1 += lL1.item()
        elH += lH.item()

        loss = lmse
        # loss = lL1
        # loss = lH
        loss.backward()

        opt.step()

    lmses.append(elmse/len(dl))
    lL1s.append(elL1/len(dl))
    lHs.append(elH/len(dl))

    print(f'ep {epoch}-> mse:{lmses[-1]:.4f}, L1:{lL1s[-1]:.4f}, H:{lHs[-1]:.4f}')

# plot losses
lmses = np.array(lmses)/np.max(lmses)
lL1s = np.array(lL1s)/np.max(lL1s)
lHs = np.array(lHs)/np.max(lHs)
plt.figure(figsize=(10, 2))
plt.plot(lmses, label='mse')
plt.yscale('log')
plt.legend()
plt.figure(figsize=(10, 2))
plt.plot(lL1s, label='L1')
plt.yscale('log')
plt.legend()
plt.figure(figsize=(10, 2))
plt.plot(lHs, label='H')
plt.yscale('log')
plt.legend()
plt.show()
print(f'train losses: mse:{lmses[-1]:.4f}, L1:{lL1s[-1]:.4f}, H:{lHs[-1]:.4f}')


# evaluate on unseen data useing mse loss
test_in = SigDS(100).data
test_out = dec(enc(SigDS(100).data))
test_loss = mse_loss(test_out, test_in)
print(f'test loss: {test_loss}')

In [ ]:
# compare input and output
plt.figure(figsize=(10, 10))
err = []
for i in range(20):
    plt.subplot(10, 2, i+1)
    x = create_random_signal()
    x̂ = dec(enc(torch.tensor(x).view(1,N_SAMPLES))).view(N_SAMPLES).detach().numpy()
    err.append(np.mean((x-x̂)**2))
    plt.plot(x, label='input')
    plt.plot(x̂, label='output')
    # plt.grid()
plt.suptitle(f'mse: {np.mean(err)}')
plt.legend()
plt.show()